# Deploying a Model from the Model Registry

In [3]:
%load_ext autoreload
%autoreload 2

import sys
CODE_FOLDER = "code"
sys.path.append(f"./{CODE_FOLDER}")

## Deploy Latest Model From Registry

Let's get the latest approved model from the Model Registry and deploy it to an endpoint.

We can use `boto3` to query the list of approved models and get the latest one. Check the [boto3 SageMaker Client API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html) for a list of everything you can do using the API.

In [9]:
from session3 import *

response = sagemaker_client.list_model_packages(
    ModelPackageGroupName=model_package_group_name,
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1,
)

package = response["ModelPackageSummaryList"][0]
package

{'ModelPackageGroupName': 'penguins',
 'ModelPackageVersion': 17,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:325223348818:model-package/penguins/17',
 'CreationTime': datetime.datetime(2023, 7, 11, 10, 30, 4, 37000, tzinfo=tzlocal()),
 'ModelPackageStatus': 'Completed',
 'ModelApprovalStatus': 'Approved'}

This is the name of the endpoint where we'll deploy the model.

In [14]:
endpoint_name = "penguins-endpoint"

Using the ARN of the model package from the Model Registry, we can deploy the model by creating a [ModelPackage](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.ModelPackage) instance and calling its `deploy()` function. The model information lives in the Model Registry, so we don't need to specify anything else.

In [11]:
from sagemaker import ModelPackage

model_package = ModelPackage(
    model_package_arn=package["ModelPackageArn"], 
    sagemaker_session=sagemaker_session,
    role=role, 
)

model_package.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1, 
    instance_type="ml.m5.large",
)

---!

## Make a few Predictions

We can create a [Predictor](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html#sagemaker.predictor.Predictor) using the endpoint name and test our model.

The payload we need to provide the model is in CSV format. Notice how the model expects data that's already transformed. We can't provide the original data from our dataset because the model will not work with it.

In [15]:
import json
import numpy as np
from sagemaker.predictor import Predictor


predictor = Predictor(endpoint_name=endpoint_name)

payload = """
0.6569590202313976, -1.0813829646495108, 1.2097102831892812, 0.9226343641317372, 1.0, 0.0, 0.0
-0.7751048801481084, 0.8822689351285553,  -1.2168066120762704, 0.9226343641317372, 0.0, 1.0, 0.0
-0.837387834894918, 0.3386660813829646, -0.26237731892812, -1.92351941317372, 0.0, 0.0, 1.0
"""

response = predictor.predict(payload, initial_args={"ContentType": "text/csv"})
response = json.loads(response.decode("utf-8"))

print(json.dumps(response, indent=2))
print(f"\nSpecies: {np.argmax(response['predictions'], axis=1)}")

{
  "predictions": [
    [
      0.112995617,
      0.0474319644,
      0.83957243
    ],
    [
      0.816265643,
      0.127040118,
      0.0566942506
    ],
    [
      0.97213167,
      0.0213899519,
      0.00647832826
    ]
  ]
}

Species: [2 0 0]


Let's now delete the endpoint.predictor.delete_endpoint()

In [16]:
predictor.delete_endpoint()